# Trend Fitness

In [40]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

In [41]:


class HybridRecommendation:
    """
        HybridRecommendation returns user's preference recommendation for fitness
        based on collaborative filtering and content-based filtering
    
    """
    def __init__(self, df):
        self.df = df

    def collaborative_filtering(self, user_id, num_recommendations=5):
        user_profile = self.df.loc[self.df['User ID'] == user_id]
        
        # Concatenate training and test data
        all_data = pd.concat([self.df[['Physical Activity Level', 'Fitness Environment']], user_profile[['Physical Activity Level', 'Fitness Environment']]])

        # Encode categorical variables
        encoder = OneHotEncoder(drop='first')  # Use drop='first' to avoid dummy variable trap
        all_features = encoder.fit_transform(all_data)
        all_features = all_features.toarray()  # Convert to dense array
        
        # Split into training and test features
        user_features = all_features[-1, :].reshape(1, -1)  # Extract user features
        all_features = all_features[:-1, :]  # Exclude user features
        
        # Compute cosine similarity
        similarities = cosine_similarity(user_features, all_features)[0]
        similar_users_indices = np.argsort(similarities)[::-1]

        # Generate recommendations
        recommendations = []
        for idx in similar_users_indices:
            similar_user_id = self.df.iloc[idx]['User ID']
            if similar_user_id != user_id:
                user_recommendations = self.df.loc[self.df['User ID'] == similar_user_id]['Primary Fitness Goal'].tolist()
                recommendations.extend(user_recommendations)
            if len(recommendations) >= num_recommendations:
                break

        return recommendations[:num_recommendations]
    
    def content_based_filtering(self, user_id, num_recommendations=5):
        user_profile = self.df.loc[self.df['User ID'] == user_id]
        user_fitness_goal = user_profile['Primary Fitness Goal'].iloc[0]

        similar_users = self.df.loc[self.df['Primary Fitness Goal'] == user_fitness_goal]
        recommendations = similar_users['Fitness Environment'].tolist()
        return recommendations[:num_recommendations]

    def hybrid_recommendations(self, user_id, num_recommendations=5):
        collaborative_recommendations = self.collaborative_filtering(user_id, num_recommendations)
        content_based_recommendations = self.content_based_filtering(user_id, num_recommendations)

        hybrid_recommendations = []
        for activity in collaborative_recommendations:
            if activity not in hybrid_recommendations:
                hybrid_recommendations.append(activity)

        for activity in content_based_recommendations:
            if activity not in hybrid_recommendations:
                hybrid_recommendations.append(activity)

        return hybrid_recommendations[:num_recommendations]
    
    

In [42]:

# Read dataset
df = pd.read_csv('fitness_dataset.csv')

# Initialize the recommendation model
hybrid_rec = HybridRecommendation(df)

# Example usage
user_id = 1
recommendations = hybrid_rec.hybrid_recommendations(user_id)
print("Hybrid Recommendations for User", user_id, ":", recommendations)

Hybrid Recommendations for User 1 : ['Injury Prevention', 'Stress Relief', 'Core Strength', 'Weight Maintenance', 'Athletic Performance']


In [53]:
# Assume new_user_data is a dictionary containing the new user's data
new_user_data = {
    'User ID': 11,
    'Primary Fitness Goal': 'Weight Loss',
    'Physical Activity Level': 'High',
    'Medical History': 'None',
    'Time Commitment': '5-7 hours per week',
    'Nutritional Preferences': 'Balanced Diet',
    'Fitness Environment': 'Gym',
    'Tracking Methods': 'Fitness App',
    'Challenges Faced': 'Lack of Motivation'
}

# Convert the dictionary to a DataFrame
new_user_df = pd.DataFrame([new_user_data])

# Instantiate HybridRecommendation class
hybrid_recommendation = HybridRecommendation(df)


# Get recommendations for the new user
recommendations = hybrid_recommendation.hybrid_recommendations(new_user_df)
print(recommendations)


   User ID Primary Fitness Goal Physical Activity Level Medical History  \
0        1          Weight Loss                Moderate             NaN   

      Time Commitment Nutritional Preferences Fitness Environment  \
0  3-5 hours per week           Balanced Diet                 Gym   

  Tracking Methods    Challenges Faced  
0      Fitness App  Lack of Motivation  
   User ID Primary Fitness Goal Physical Activity Level Medical History  \
0       11          Weight Loss                    High            None   

      Time Commitment Nutritional Preferences Fitness Environment  \
0  5-7 hours per week           Balanced Diet                 Gym   

  Tracking Methods    Challenges Faced  
0      Fitness App  Lack of Motivation  


C:\Users\HP\AppData\Local\Temp\ipykernel_908\1866277136.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(new_user_df[df['User ID'] == user_id])


ValueError: Operands are not aligned. Do `left, right = left.align(right, axis=1, copy=False)` before operating.